<a href="https://colab.research.google.com/github/MinKimIP/IPA-public/blob/master/ipos-tm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get IPOS Trade Mark Data


In [0]:
%%capture
!pip install pandas --upgrade

In [1]:
# import necessary packages
import numpy as np
import pandas as pd
import requests
from datetime import timedelta, date
print(f"pandas {pd.__version__}")

pandas 0.25.3


In [0]:
# define required custom functions

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)
    return next_month - timedelta(days=next_month.day)


# define function to get Singaporean Trade Mark data

def getIPOSmonth(year, month):
    # This part will obtain the daily JSON data, concatenate for the month, then save as a JSON file.
    # Each row represents the data relating to a single Trade Mark application.
    url_base = "https://api.data.gov.sg/v1/technology/ipos/trademarks?lodgement_date="
    start_date = date(year, month, 1)
    end_date = last_day_of_month(start_date)
    for single_date in daterange(start_date, end_date):
        lodgement_date = single_date.strftime("%Y-%m-%d")
        url = url_base+lodgement_date
        response = requests.get(url)
        data = response.json()
        temp = pd.DataFrame(data['items'])
        temp['lodgementDate'] = lodgement_date    
        try:
            df
        except:
            df = temp
        else:
            df = df.append(temp, sort=True)
    df.to_json(f'{year}-{month}.json', orient='records')
    del df


def getIPOSdata(year, month, skip_existing=True):
    """
    Obtains JSON data from the Intellectual Property Office of Singapore through the Open Data API
    """
    try:
        pd.read_json(f'{year}-{month}.json', orient='records')
    except:
        getIPOSmonth(year, month)
    else:
        if skip_existing==True:
            print(f"Skipping {year}-{month} because the JSON already exists")
        else:
            getIPOSmonth(year, month)

In [0]:
years = range(2020, 2019, -1)
months = range(3, 0, -1)

for year in years:
    for month in months:
        getIPOSdata(year, month)